In [1]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> ""
## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [2]:
# Setting a temp directory for tests
TMP=./tmp/test-py-commands
## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..
alias | wc -l

8


In [3]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3443)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3443

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/home/test3513/tom-project/shell-scripts/tests/tmp/test-py-commands/test-3443


In [4]:
export PYTHON_CMD="/usr/bin/time python -u"
export PYTHON="$NICE $PYTHON_CMD"
export PYTHONPATH="$HOME/python:$PYTHONPATH"

In [5]:
## add-python-path(pkg-dir): add PKG-DIR to PATH and PARENT to PYTHONPATH

function add-python-path () {
    local package_path="$1"
    local parent_path=$(realpath "$package_path/..")
    prepend-path "$package_path"
    export PYTHONPATH="$parent_path:$PYTHONPATH"
}

alias mezcla-devel='add-python-path $HOME/python/Mezcla/mezcla'
alias mezcla-main='add-python-path $HOME/python/Mezcla-main/mezcla'
alias mezcla-tom='add-python-path $HOME/python/Mezcla-tom/mezcla'

if [[ ! "$PATH" =~ mezcla ]]; then
    true
fi

In [6]:
## FUNCTIONS REQUIRED FOR SMOOTH RUNNING OF THE TESTS
function ps-all () { 
    local pattern="$1";
    local pager=cat;
    if [ "$pattern" = "" ]; then 
        pattern="."; 
        pager=$PAGER
    fi;
    ps_mine.sh --all | $EGREP -i "((^USER)|($pattern))" | $pager;
    }

function show-path-dir () { (echo "${1}:"; printenv "$1" | perl -pe "s/:/\n/g;") | $PAGER; }
alias show-path='show-path-dir PATH'

function append-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="${PATH}:$1"; fi }
function prepend-path () { if [[ ! (($PATH =~ ^$1:) || ($PATH =~ :$1:) || ($PATH =~ :$1$)) ]]; then export PATH="$1:${PATH}"; fi }

function append-python-path () { export PYTHONPATH=${PYTHONPATH}:"$1"; }
function prepend-python-path () { export PYTHONPATH="$1":${PYTHONPATH}; }
 

In [7]:

add-python-path $temp_dir
alias ps-python-full='ps-all python'

alias ps-python='ps-python-full | $EGREP -iv "(screenlet|ipython|egrep|update-manager|software-properties|networkd-dispatcher)"'
alias show-python-path='show-path-dir PYTHONPATH'

realpath: ./tmp/test-py-commands/test-3443/..: No such file or directory


In [8]:
## ERROR 1 - process ID list syntax error

# $ ps-python-full
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).

In [9]:
## ERROR 2 - bash : command not found and process ID list syntax error
# $ ps-python
# | bash: -iv: command not found
# | error: process ID list syntax error
# | 
# | Usage:
# |  ps [options]
# | 
# |  Try 'ps --help <simple|list|output|threads|misc|all>'
# |   or 'ps --help <s|l|o|t|m|a>'
# |  for additional help text.
# | 
# | For more details see ps(1).


In [10]:
## show-python-path DISPLAYS ALL THE AVAILABLE PATHS OF PYTHON
show-python-path

PYTHONPATH:

/home/test3513/python



In [11]:
function python-lint-work() { python-lint-full "$@" 2>&1 | $EGREP -v '\((bad-continuation|bad-option-value|fixme|invalid-name|locally-disabled|too-few-public-methods|too-many-\S+|trailing-whitespace|star-args|unnecessary-pass)\)' | $EGREP -v '^(([A-Z]:[0-9]+)|(Your code has been rated)|(No config file found)|(\-\-\-\-\-))' | $PAGER; }

function python-lint() { python-lint-work "$@" 2>&1 | $EGREP -v '(Exactly one space required)|\((bad-continuation|bad-whitespace|bad-indentation|bare-except|c-extension-no-member|consider-using-enumerate|consider-using-with|global-statement|global-variable-not-assigned|keyword-arg-before-vararg|len-as-condition|line-too-long|logging-not-lazy|misplaced-comparison-constant|missing-final-newline|redefined-variable-type|redundant-keyword-arg|superfluous-parens|too-many-arguments|too-many-instance-attributes|trailing-newlines|useless-\S+|wrong-import-order|wrong-import-position)\)' | $PAGER; }


In [12]:
## ERROR GENERATED : -v COMMAND NOT FOUND
# $ python-lint-work $BIN_DIR/tests/batspp.py
# | bash: -v: command not found

In [13]:
## ERROR GENERATED : -v COMMAND NOT FOUND
# $ python-lint $BIN_DIR/tests/batspp.py
# | bash: -v: command not found

In [14]:
# (REQUIRED BY run-python-lint-batched)

function downcase-stdin { perl -pe "use open ':std', ':encoding(UTF-8)'; s/.*/\L$&/;"; }
function downcase-text() { echo "$@" | downcase-stdin; }

function todays-date() { date '+%d%b%y' | downcase-stdin; }

function get-python-lint-dir () {
    local python_version_major=$(pylint --version 2>&1 | extract_matches.perl "Python (\d)")
    local affix="py${python_version_major}"
    local out_dir="_pylint/$(todays-date)-$affix"
    echo "$out_dir"
}

function run-python-lint-batched () {
    local file_spec="$@"
    if [ "$file_spec" = "" ]; then file_spec="*.py"; fi

    local out_dir=$(get-python-lint-dir)
    mkdir -p "$out_dir"

    (for f in $($LS $file_spec); do

         local b=$(basename "$f")
         local pre=""

         if [[ $f =~ / ]]; then pre="$(basename $(dirname "$f"))-"; fi
         DEBUG_LEVEL=5 python-lint "$f" >| "$out_dir/$pre$b".log 2>&1
         head "$out_dir/$pre$b".log
     done) >| "$out_dir/summary.log"
    less -p '^\** Module' "$out_dir/summary.log";
}

In [15]:
# ERROR: Pattern not found  (press RETURN)
# run-python-lint-batched ./print.py 

In [16]:
# python-import-path(module): find path for package directory of MODULE

function python-import-path-all() { local module="$1"; python -u -v -c "import $module" 2>&1; }
function python-import-path-full() { local module="$1"; python-import-path-all "$@" | extract_matches.perl "((matches (.*\W$module[^/]*[/\.][^/]*))|ModuleNotFoundError)"; }
function python-import-path() { python-import-path-full "$@" | head -1; }

# mezcla MODULE IS USED FOR TEST

python-import-path 'mezcla'
linebr
python-import-path-full 'mezcla'
linebr
# WORKS WELL, SHOWS SOME ISSUES ON BATSPP 2.1.X
python-import-path-all 'mezcla' | grep 'mezcla' | head -n 5

extract_matches.perl: command not found
--------------------------------------------------------------------------------
extract_matches.perl: command not found
--------------------------------------------------------------------------------
# /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/__init__.cpython-39.pyc matches /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__init__.py
# code object from '/home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/__init__.cpython-39.pyc'
# /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/debug.cpython-39.pyc matches /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/debug.py
# code object from '/home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/debug.cpython-39.pyc'
# /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/sys_version_info_hack.cpython-39.pyc matches /home/test3513/miniconda3/lib/python3.9/site-packag

In [17]:
# RETURNS A LIST OF PYTHON MODULES
rm -rf ./*
pip3 freeze

asttokens==2.1.0
backcall==0.2.0
brotlipy==0.7.0
certifi==2021.10.8
cffi @ file:///opt/conda/conda-bld/cffi_1642701102775/work
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
colorama @ file:///tmp/build/80754af9/colorama_1607707115595/work
conda==4.12.0
conda-content-trust @ file:///tmp/build/80754af9/conda-content-trust_1617045594566/work
conda-package-handling @ file:///tmp/build/80754af9/conda-package-handling_1649105784853/work
cryptography @ file:///tmp/build/80754af9/cryptography_1639414572950/work
decorator==5.1.1
executing==1.2.0
idna @ file:///tmp/build/80754af9/idna_1637925883363/work
ipython==8.6.0
jedi==0.18.1
matplotlib-inline==0.1.6
mezcla==1.3.2
parso==0.8.3
pexpect==4.8.0
pickleshare==0.7.5
prompt-toolkit==3.0.32
ptyprocess==0.7.0
pure-eval==0.2.2
pycosat==0.6.3
pycparser @ file:///tmp/build/80754af9/pycparser_1636541352034/work
Pygments==2.13.0
pyOpenSSL @ file:///opt/conda/conda-bld/pyopenssl_1643788558760/work
PySocks @ file:///

In [18]:
# note: gotta hate python!
function python-module-version-full { local module="$1"; python -c "import $module; print([v for v in [getattr($module, a, '') for a in '__VERSION__ VERSION __version__ version'.split()] if v][0])"; }

function python-module-version { python-module-version-full "$@" 2> /dev/null; }
function python-package-members() { local package="$1"; python -c "import $package; print(dir($package));"; }
#
alias python-setup-install='log=setup.log;  rename-with-file-date $log;  uname -a > $log;  python setup.py install --record installed-files.list >> $log 2>&1;  ltc $log'

alias python-uninstall-setup='cat installed-files.list | xargs /bin/rm -vi; rename_files.perl -regex ^ un installed-files.list'

# MODULE USED: mezcla
python-module-version-full mezcla
linebr
python-module-version mezcla
linebr
python-package-members mezcla
linebr

1.3.2
--------------------------------------------------------------------------------
1.3.2
--------------------------------------------------------------------------------
['PYTHON3_PLUS', 'TL', '__VERSION__', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'debug', 'gh', 'glue_helpers', 'mezcla', 'sys', 'sys_version_info_hack', 'system', 'tpo_common']
--------------------------------------------------------------------------------


In [19]:
## rename-with-file-date REQUIRED
function rename-with-file-date() {

    local f new_f
    local move_command="move"
    if [ "$1" = "--copy" ]; then

        move_command="command cp --interactive --verbose --preserve"
        shift
    fi
    for f in "$@"; do
    
        if [ -e "$f" ]; then
           new_f=$(get-free-filename "$f".$(date --reference="$f" '+%d%b%y') ".")

           eval "$move_command" "$f" "$new_f";
        fi
    done;

}

function get-free-filename() {
    local base="$1"
    local sep="$2"
    local L=1
    local filename="$base"
    while [ -e "$filename" ]; do
        let L++
        filename="$base$sep$L"
    done;
    echo "$filename"
}

alias move='mv'
export PAGER=less
export PAGER_CHOPPED="less -S"
export PAGER_NOEXIT="less -+F"
function zless () { zcat "$@" | $PAGER; }

function zhead () {
    local file="$1"
    shift
    zcat "$file" | head "$@"
}
alias less-='$PAGER_NOEXIT'
alias less-clipped='$PAGER_NOEXIT -S'
alias less-tail='$PAGER_NOEXIT +G'
alias less-tail-clipped='$PAGER_NOEXIT +G -S'
alias ltc=less-tail-clipped


In [20]:
## REQUIRES setup.py (TBC)

# python-setup-install
# linebr
# python-uninstall-setup

In [21]:
function python-import-path-all() { local module="$1"; python -u -v -c "import $module" 2>&1; }
function python-import-path-full() { local module="$1"; python-import-path-all "$@" | extract_matches.perl "((matches (.*\W$module[^/]*[/\.][^/]*))|ModuleNotFoundError)"; }
function python-import-path() { python-import-path-full "$@" | head -1; }

In [22]:
# mezcla MODULE IS USED FOR TEST

python-import-path 'mezcla'
linebr
python-import-path-full 'mezcla'
linebr

# WORKS WELL, SHOWS SOME ISSUES ON BATSPP 2.1.X
python-import-path-all 'mezcla' | grep 'mezcla' | head -n 5

extract_matches.perl: command not found
--------------------------------------------------------------------------------
extract_matches.perl: command not found
--------------------------------------------------------------------------------
# /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/__init__.cpython-39.pyc matches /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__init__.py
# code object from '/home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/__init__.cpython-39.pyc'
# /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/debug.cpython-39.pyc matches /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/debug.py
# code object from '/home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/debug.cpython-39.pyc'
# /home/test3513/miniconda3/lib/python3.9/site-packages/mezcla/__pycache__/sys_version_info_hack.cpython-39.pyc matches /home/test3513/miniconda3/lib/python3.9/site-packag

In [23]:
# RETURNS A LIST OF PYTHON MODULES
rm -rf ./*
pip3 freeze | head

asttokens==2.1.0
backcall==0.2.0
brotlipy==0.7.0
certifi==2021.10.8
cffi @ file:///opt/conda/conda-bld/cffi_1642701102775/work
charset-normalizer @ file:///tmp/build/80754af9/charset-normalizer_1630003229654/work
colorama @ file:///tmp/build/80754af9/colorama_1607707115595/work
conda==4.12.0
conda-content-trust @ file:///tmp/build/80754af9/conda-content-trust_1617045594566/work
conda-package-handling @ file:///tmp/build/80754af9/conda-package-handling_1649105784853/work


In [24]:
function python-module-version-full { local module="$1"; python -c "import $module; print([v for v in [getattr($module, a, '') for a in '__VERSION__ VERSION __version__ version'.split()] if v][0])"; }
function python-module-version { python-module-version-full "$@" 2> /dev/null; }
function python-package-members() { local package="$1"; python -c "import $package; print(dir($package));"; }

alias python-setup-install='log=setup.log;  rename-with-file-date $log;  uname -a > $log;  python setup.py install --record installed-files.list >> $log 2>&1;  ltc $log'
alias python-uninstall-setup='cat installed-files.list | xargs /bin/rm -vi; rename_files.perl -regex ^ un installed-files.list'

In [25]:
# MODULE USED: mezcla
python-module-version-full mezcla
linebr
python-module-version mezcla
linebr
python-package-members mezcla
linebr

1.3.2
--------------------------------------------------------------------------------
1.3.2
--------------------------------------------------------------------------------
['PYTHON3_PLUS', 'TL', '__VERSION__', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'debug', 'gh', 'glue_helpers', 'mezcla', 'sys', 'sys_version_info_hack', 'system', 'tpo_common']
--------------------------------------------------------------------------------


In [26]:
## rename-with-file-date REQUIRED
function rename-with-file-date() {
    local f new_f
    local move_command="move"
    if [ "$1" = "--copy" ]; then

        move_command="command cp --interactive --verbose --preserve"
        shift
    fi
    for f in "$@"; do
    
        if [ -e "$f" ]; then
           new_f=$(get-free-filename "$f".$(date --reference="$f" '+%d%b%y') ".")

           eval "$move_command" "$f" "$new_f";
        fi
    done;

}

function get-free-filename() {
    local base="$1"
    local sep="$2"
    local L=1
    local filename="$base"
    while [ -e "$filename" ]; do
        let L++
        filename="$base$sep$L"
    done;
    echo "$filename"
}

alias move='mv'
export PAGER=less
export PAGER_CHOPPED="less -S"
export PAGER_NOEXIT="less -+F"
function zless () { zcat "$@" | $PAGER; }

function zhead () {
    local file="$1"
    shift
    zcat "$file" | head "$@"
}

alias less-='$PAGER_NOEXIT'
alias less-clipped='$PAGER_NOEXIT -S'
alias less-tail='$PAGER_NOEXIT +G'
alias less-tail-clipped='$PAGER_NOEXIT +G -S'
alias ltc=less-tail-clipped

In [27]:
## REQUIRES setup.py (TBC)

# python-setup-install
# linebr
# python-uninstall-setup

In [28]:
alias set-xterm-window='$BIN_DIR/set_xterm_title.bash'
# alias set-xterm-window='set_xterm_title.bash'
function ipython() { 
    local ipython=$(which ipython)
    if [ "$ipython" = "" ]; then echo "Error: install ipython first"; return; fi
    set-xterm-window "ipython [$PWD]"
    $ipython '$@'
}

function python-trace {
    local script="$1"
    shift
    $PYTHON -m trace --trace $(which "$script") "$@"
    }
    
ipython --version | head
linebr
# STATEMENT TRACING WORKS WELL
python-trace l $BIN_DIR/tests/auto_batspp.py | head

/home/test3513/tom-project/shell-scripts/tests/../set_xterm_title.bash: line 239: /_set_xterm_title.11912.full.list: Permission denied
/home/test3513/tom-project/shell-scripts/tests/../set_xterm_title.bash: line 240: /_set_xterm_title.11912.icon.list: Permission denied
[TerminalIPythonApp] WARNING | File '$@' doesn't exist
]1;ipython [/home/test3513/tom-project/shell-scripts/tests/tmp/test-py-commands/test-3443]]2;ipython [/home/test3513/tom-project/shell-scripts/tests/tmp/test-py-commands/test-3443]--------------------------------------------------------------------------------
 --- modulename: auto_batspp, funcname: <module>
auto_batspp.py(6): """ 
auto_batspp.py(14): import os
auto_batspp.py(15): import sys
Cannot run file '/home/test3513/tom-project/shell-scripts/tests/../tests/auto_batspp.py' because: [Errno 2] No such file or directory: './batspp-only/'
auto_batspp.py(17): IPYNB = ".ipynb"
auto_batspp.py(18): BATSPP = ".batspp"
auto_batspp.py(19): BATS = ".bats"
auto_batspp.p

In [29]:
# which-python SHOWS THE PATH OF PYTHON
alias which-python='which python'
which-python
#(ADDED FOR PYTHON3)
alias which-py3='which python3' 
which-py3

/home/test3513/miniconda3/bin/python
/home/test3513/miniconda3/bin/python3


In [30]:
function py-diff () { do_diff.sh '*.py *.mako' "$@" 2>&1 | $PAGER; }

In [31]:
rm -rf ./*
printf "print('THIS IS A TEST')" > atest.py
printf "print('THIS IS A TEST')" > xyz.py
printf "print('THIS IS A TEST11')" > abc1.py

# SHOWS ALL THE DIFFERENCES BETWEEN THE PYTHON SCRIPTS
# WORKS FINE - HALTS TESTS

# $ py-diff atest.py xyz.py

In [32]:
# TEST NOT APPLICABLE RN 
function kivy-win32-env {
   export PYTHONPATH='c:/cartera-de-tomas/python;c:/Program-Misc/python/kivy-1-9-0/kivy27'
   kivy_dir="/c/Program-Misc/python/kivy-1-9-0"
   python_dir="$kivy_dir/Python27"
   prepend-path "$kivy_dir:$kivy_dir/Python27:$kivy_dir/tools:$kivy_dir/Python27/Scripts:$kivy_dir/gstreamer/bin:$kivy_dir/MinGW/bin:$kivy_dir/SDL2/bin"
}

In [33]:
alias elide-data='python -m transpose_data --elide'
alias kill-python="kill_em.sh --filter 'ipython|emacs' python"
alias kill-python-all="kill_em.sh python"

In [34]:
# # ERRORS = OSTYPE is 'undefined variable' 
# # NOTE: elide-data shows RuntimeError if any testfile is named abc.py

# $ echo $OSTYPE
#   linebr
#   elide-data $BIN_DIR/template.py
#   linebr
#   kill-python
#   linebr
#   kill-python-all

# | linux-gnu
# | --------------------------------------------------------------------------------
# | /home/xaea12/miniconda3/bin/python: No module named transpose_data
# | --------------------------------------------------------------------------------
# | pattern=/:[0-9][0-9] [^ ]*python/
# | filter=/ipython|emacs/
# | OSTYPE: Undefined variable.
# | --------------------------------------------------------------------------------
# | pattern=/:[0-9][0-9] [^ ]*python/
# | filter=/($)(^)/
# | OSTYPE: Undefined variable.


In [35]:
alias TODAY='date'
alias extract-matches='extract_matches.perl'

function run-jupyter-notebook () {
    local port="$1"; if [ "$port" = "" ]; then port=8888; fi
    local ip="$2"; if [ "$ip" = "" ]; then ip="127.0.0.1"; fi
    local log="$TEMP/jupyter-$(TODAY).log"
    jupyter notebook --NotebookApp.token='' --no-browser --port $port --ip $ip >> "$log" 2>&1 &
    echo "$log"
    local delay=5
    echo "sleeping $delay seconds for log to stabalize (effing jupyter)"
    sleep $delay
    tail "$log"
    echo -n "URL: "
    extract-matches 'http:\S+' "$log" | sort -u
}
alias jupyter-notebook-redir=run-jupyter-notebook
alias jupyter-notebook-redir-open='run-jupyter-notebook 8888 0.0.0.0'

In [36]:
# # ERROR GENERATED (see below)
# $ jupyter-notebook-redir-open

# | [1] 5335
# | /jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log
# | sleeping 5 seconds for log to stabalize (effing jupyter)
# | bash: /jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log: Permission denied
# | [1]+  Exit 1                  jupyter notebook --NotebookApp.token='' --no-browser --port $port --ip $ip >> "$log" 2>&1
# | tail: cannot open '/jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log' for reading: No such file or directory
# | URL: Can't open /jupyter-Sun Aug 28 08:13:40 AM +0545 2022.log: No such file or directory at /home/xaea12/bin/extract_matches.perl line 132.


In [37]:
function extract-text() { python -m extract_document_text "$@"; }
alias xtract-text='extract-text'

In [38]:
# # ERROR - MODULE NOT FOUND

# $ rm -rf ./*
#   printf "print('THIS IS A TEST FOR TEXT EXTRACT ALIAS.')" > textex.py
#   xtract-text ./textex.py

# | /home/xaea12/miniconda3/bin/python: No module named extract_document_text

In [39]:
function test-script () {
    local base=$(basename "$1" .py)
    local date=$(todays-date)
    local root=$(hg root)
    PYTHONPATH="$root:.:$SANDBOX/tests:$PYTHONPATH" $NICE $PYTHON tests/test_$base.py --verbose >| tests/_test_$base.$date.log 2>&1;
    less-tail tests/_test_$base.$date.log;
}

alias test-script-debug='ALLOW_SUBCOMMAND_TRACING=1 DEBUG_LEVEL=5 MISC_TRACING_LEVEL=5 test-script'

In [40]:
# # ERROR (LOG FILE NOT FOUND)
# $ test-script ./textex.py
# | abort: no repository found in '/tmp/test-py-commands/test-3443' (.hg not found)
# | bash: tests/_test_textex.03sep22.log: No such file or directory
# | tests/_test_textex.03sep22.log: No such file or directory

In [41]:
function randomize-datafile() {
    local file="$1"
    local num_lines="$2"
    if [ "$num_lines" = "" ]; then num_lines=$(wc -l "$file"); fi
    
    head -1 "$file"
    tail --lines=+2 "$file" | python -m randomize_lines | head -$num_lines
}

In [42]:
# TESTING A PYTHON SCRIPT
printf "print('THIS IS A TEST10')\nprint('THIS IS A TEST11')\nprint('THIS IS A TEST12')\nprint('THIS IS A TEST13')" > random_line_test.py

In [43]:
# ERROR : MISSING MODULE (No module named randomize_lines)
# $ randomize-datafile random_line_test.py
# | print('THIS IS A TEST10')
# | print('THIS IS A TEST11')
# | print('THIS IS A TEST12')
# | /home/xaea12/miniconda3/bin/python: No module named randomize_lines

In [44]:
function filter-random() {
    local pct="$1"
    local file="$2"
    local include_header="$3"
    if [ "$include_header" == "" ]; then include_header=1; fi

    local ratio=$(perl -e "printf('%.3f', ($pct / 100.0));")
    local compressed=0
    if [[ $file =~ .gz ]]; then compressed=1; fi
    local dir=$(dirname $file)
    local base=$(basename $file)
    local type="cat"
    local result="$dir/_r${pct}pct-$base"


    if [ "$compressed" = "1" ]; then 
       type="zcat"; 
       result=$(echo "$result" | perl -pe 's/.gz$//;')
    fi
    local opts=""
    if [ "$include_header" = "1" ]; then opts="$opt --include-header"; fi
    $type "$file" | $PYTHON -m filter_random $opts --ratio $ratio - > "$result" 2> '$result.log'

    if [ "$compressed" = "1" ]; then 
       gzip --force "$result"; 
    fi
}

In [45]:
# filter-random _r10pct-fubar.data (TBF - NEED HELP OPERATING)

In [46]:
function conditional-source () { if [ -e "$1" ]; then source "$1"; else echo "Warning: bash script file not found (so not sourced):"; echo "    $1"; fi; }
function quiet-unalias {

    if [ "$BATS_TEST_FILENAME" != "" ]; then
        if [ "$BATCH_MODE" != "1" ]; then
            echo "Ignoring unalias over $* for sake of bats"
        fi
        return
    fi
    unalias "$@" 2> /dev/null || true;
}

conditional-source $BIN_DIR/anaconda-aliases.bash
conditional-source $BIN_DIR/git-aliases.bash

In [47]:
function curl-dump () {
    local url="$1";
    local base=$(basename $url);
    curl $url > $base;
}

function url-path () {
    local file="$1"
    echo $(realpath "$file") | perl -pe 's@^@file:///@;'
}


In [48]:
# TESTS FOR curl-dump AND url-path
curl-dump https://www.example.com/
linebr
ls -l
linebr
url-path $BIN_DIR/tests/auto_batspp.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1256  100  1256    0     0    935      0  0:00:01  0:00:01 --:--:--   935
--------------------------------------------------------------------------------
total 20
-rw-rw-r-- 1 test3513 test3513   25 Nov  4 20:56 abc1.py
-rw-rw-r-- 1 test3513 test3513   23 Nov  4 20:56 atest.py
-rw-rw-r-- 1 test3513 test3513  103 Nov  4 20:56 random_line_test.py
-rw-rw-r-- 1 test3513 test3513 1256 Nov  4 20:56 www.example.com
-rw-rw-r-- 1 test3513 test3513   23 Nov  4 20:56 xyz.py
--------------------------------------------------------------------------------
file:////home/test3513/tom-project/shell-scripts/tests/auto_batspp.py


In [49]:
function invoke-browser() {
    local browser_executable="$1"
    local file="$2"
    if [ "$file" != "" ]; then
        if [[ ! $file =~ http ]]; then
            file=$(url-path "$file")
        fi
    fi
    local browser_base=$(basename "$browser_executable")
    $browser_executable "$file" >> $TEMP/$browser_base.$(TODAY).log 2>&1 &
}

function which { command which "$1" 2> /dev/null; }
alias firefox='invoke-browser "'"$(which firefox)"'"'


In [50]:
# # WORKS WELL - HALTS TESTING
# $ firefox
# | [1] 15869

In [51]:
pwd
rm -rf *

/home/test3513/tom-project/shell-scripts/tests/tmp/test-py-commands/test-3443
